                                            Baseline+improvement2 ( Edge Aware Mirror Network)

In [1]:
!pip install torch-lr-finder 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.6 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.9.90
    Uninstalling nvidia-curand-cu12-1

In [4]:
!git clone https://github.com/sdy1999/EAMNet.git /kaggle/working/EAMNet

Cloning into '/kaggle/working/EAMNet'...
remote: Enumerating objects: 93, done.
remote: Counting objects: 100% (93/93), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 93 (delta 42), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (93/93), 45.29 KiB | 3.77 MiB/s, done.
Resolving deltas: 100% (42/42), done.


In [2]:
# =============================================================================
# Plant Segmentation – 6-channel (RGB + CEI + ExR + Sobel) · DeepLabV3+ ·
# ResNet-101-DO-SE backbone · Cosine-Warm-Restarts LR
# =============================================================================
import os, gc, glob, shutil, random, math, cv2, numpy as np
import torch, torch.nn as nn, torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
from torch.optim      import AdamW
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from torch.optim.swa_utils    import AveragedModel
from sklearn.model_selection  import train_test_split
from torchmetrics.classification import (
    MulticlassJaccardIndex, MulticlassPrecision,
    MulticlassRecall, MulticlassF1Score
)
import albumentations as A

# -----------------------------------------------------------------------------
# 0) ENV & RNG
# -----------------------------------------------------------------------------
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:64,expandable_segments:True"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("▶ device:", device)

# -----------------------------------------------------------------------------
# 1) DATA SPLIT  (PhenoBench 70 : 30)
# -----------------------------------------------------------------------------
BASE  = '/kaggle/input/phenobench/PhenoBench'
IMG_D = f'{BASE}/train/images'
MSK_D = f'{BASE}/train/semantics'
WRK   = '/kaggle/working/train_split'
TR,VA = f'{WRK}/train', f'{WRK}/val'
for d in (TR, VA):
    os.makedirs(f'{d}/images',   exist_ok=True)
    os.makedirs(f'{d}/semantics', exist_ok=True)

imgs = [p for p in sorted(glob.glob(f'{IMG_D}/*'))
        if os.path.exists(f"{MSK_D}/{os.path.basename(p)}")]

tr_files, va_files = train_test_split(imgs, test_size=.30, random_state=SEED)

def _remap(m): m = m.astype(np.uint8); m[m==3] = 1; m[m==4] = 1; return m
def _copy(src_lst, dst_root):
    for p in src_lst:
        fn = os.path.basename(p)
        shutil.copy(p, f'{dst_root}/images/{fn}')
        m = cv2.imread(f'{MSK_D}/{fn}', -1)
        cv2.imwrite(f'{dst_root}/semantics/{fn}', _remap(m))
_copy(tr_files, TR); _copy(va_files, VA)
print(len(tr_files), "train   |", len(va_files), "val")

# -----------------------------------------------------------------------------
# 2) CLASS WEIGHTS  (pixel-freq inverse √)
# -----------------------------------------------------------------------------
pix = np.zeros(3, np.int64)
for mp in glob.glob(f'{TR}/semantics/*'):
    u, c = np.unique(cv2.imread(mp, 0), return_counts=True)
    for ui, ci in zip(u, c): pix[int(ui)] += ci
wts = (pix.sum() / (3 * pix)).astype(np.float32)
CLS_WT = torch.tensor(wts, device=device)
print("class-weights:", wts.round(3).tolist())

# -----------------------------------------------------------------------------
# 3) EXTRA CHANNELS  (CEI / ExR / Sobel)
# -----------------------------------------------------------------------------
def CEI(img):
    B, G, R = cv2.split(img.astype(np.float32))
    exg = 2*G - R - B
    return cv2.normalize(exg, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

def ExR(img):
    B, G, R = cv2.split(img.astype(np.float32))
    exr = 1.4*R - G
    return cv2.normalize(exr, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

def SobelMag(img):
    g = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    sx = cv2.Sobel(g, cv2.CV_32F, 1, 0, 3)
    sy = cv2.Sobel(g, cv2.CV_32F, 0, 1, 3)
    mag = np.sqrt(sx*sx + sy*sy)
    return cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

# -----------------------------------------------------------------------------
# 4) AUGMENT + DATASET
# -----------------------------------------------------------------------------
SIZE = (1024, 1024)
train_aug = A.Compose([
    A.RandomScale(0.25, p=.5),
    A.PadIfNeeded(SIZE[0], SIZE[1], border_mode=cv2.BORDER_CONSTANT,
                  value=0, mask_value=0, p=1.0),
    A.RandomCrop(*SIZE, p=1.0),
    A.HorizontalFlip(.5), A.VerticalFlip(.5),
    A.Rotate(30, border_mode=cv2.BORDER_CONSTANT, value=0, mask_value=0, p=.5),
    A.RandomBrightnessContrast(.2, .2, p=.5),
    A.HueSaturationValue(15, 25, 15, p=.5)
], additional_targets={'mask': 'mask'})

class ResizeTF:
    def __call__(self, x, m):
        x = F.interpolate(x[None], SIZE, mode='bilinear',
                          align_corners=False)[0]
        m = F.interpolate(m[None, None].float(), SIZE,
                          mode='nearest')[0, 0].long()
        return x, m

class PhenoBench(Dataset):
    def __init__(self, root, aug=None, tf=None, cutmix=False, p=0.2):
        self.imgs = sorted(glob.glob(f'{root}/images/*'))
        self.mskd = f'{root}/semantics'
        self.aug, self.tf, self.cutmix, self.p = aug, tf, cutmix, p
    def __len__(self): return len(self.imgs)
    def _load(self, idx):
        fn = os.path.basename(self.imgs[idx])
        return cv2.imread(self.imgs[idx]), cv2.imread(f'{self.mskd}/{fn}', 0)
    def _to_tensor(self, im):
        R, G, B = cv2.split(im)
        ch = np.stack([R, G, B, CEI(im), ExR(im), SobelMag(im)], 0) / 255.0
        ch = (ch - .5) / .5
        return torch.tensor(ch, dtype=torch.float32)
    def __getitem__(self, idx):
        im, ma = self._load(idx)
        if self.aug:
            d = self.aug(image=im, mask=ma); im, ma = d['image'], d['mask']
        # simple CutMix square
        if self.cutmix and random.random() < self.p:
            im2, ma2 = self._load(random.randrange(len(self)))
            if self.aug:
                d2 = self.aug(image=im2, mask=ma2); im2, ma2 = d2['image'], d2['mask']
            h, w = ma.shape; bh, bw = h//4, w//4
            y0, x0 = random.randint(0, h-bh), random.randint(0, w-bw)
            im[y0:y0+bh, x0:x0+bw] = im2[y0:y0+bh, x0:x0+bw]
            ma[y0:y0+bh, x0:x0+bw] = ma2[y0:y0+bh, x0:x0+bw]
        x = self._to_tensor(im)
        m = torch.tensor(ma, dtype=torch.long)
        if self.tf: x, m = self.tf(x, m)
        return x, m

train_ds = PhenoBench(TR, train_aug, ResizeTF(), cutmix=True, p=.2)
val_ds   = PhenoBench(VA, None,       ResizeTF())

# -----------------------------------------------------------------------------
# 5) LOSSES
# -----------------------------------------------------------------------------
class Dice(nn.Module):
    def __init__(self, eps=1e-6): super().__init__(); self.eps = eps
    def forward(self, l, t):
        p = F.softmax(l, 1)
        t1 = F.one_hot(t, 3).permute(0, 3, 1, 2).float()
        i  = (p * t1).sum((0,2,3)); u = (p + t1).sum((0,2,3))
        return 1 - ((2*i + self.eps) / (u + self.eps)).mean()

class Focal(nn.Module):
    def __init__(self, g=2.0, a=.25): super().__init__(); self.g, self.a = g, a
    def forward(self, l, t):
        ce = F.cross_entropy(l, t, reduction='none')
        pt = torch.exp(-ce)
        return (self.a * (1-pt)**self.g * ce).mean()

CE   = nn.CrossEntropyLoss(weight=CLS_WT)
DICE = Dice(); FOC = Focal()
def loss_fn(l, t): return CE(l, t) + .5*DICE(l, t) + .25*FOC(l, t)

# -----------------------------------------------------------------------------
# 6) SE Block (unchanged)
# -----------------------------------------------------------------------------
class SEBlock(nn.Module):
    def __init__(self, ch, r=16):
        super().__init__()
        self.fc1 = nn.Conv2d(ch, ch//r, 1)
        self.fc2 = nn.Conv2d(ch//r, ch, 1)
    def forward(self, x):
        w = F.adaptive_avg_pool2d(x, 1)
        w = F.relu(self.fc1(w), inplace=True)
        w = torch.sigmoid(self.fc2(w))
        return x * w

# -----------------------------------------------------------------------------
# 7) BACKBONE + HEAD (ResNet-101-DO-SE)  **unchanged names**
# -----------------------------------------------------------------------------
def conv3(ic, oc, s=1, d=1): return nn.Conv2d(ic, oc, 3, s, padding=d, dilation=d, bias=False)
def conv1(ic, oc, s=1):      return nn.Conv2d(ic, oc, 1, s, bias=False)

class Bottleneck(nn.Module):
    expansion = 4
    def __init__(self, inplanes, planes, stride=1, downsample=None, dilation=1):
        super().__init__()
        self.conv1, self.bn1 = conv1(inplanes, planes), nn.BatchNorm2d(planes)
        self.conv2, self.bn2 = conv3(planes, planes, stride, dilation), nn.BatchNorm2d(planes)
        self.conv3, self.bn3 = conv1(planes, planes*4), nn.BatchNorm2d(planes*4)
        self.downsample = downsample
        self.se = SEBlock(planes*4)
    def forward(self, x):
        idt = x
        out = F.relu(self.bn1(self.conv1(x)), inplace=True)
        out = F.relu(self.bn2(self.conv2(out)), inplace=True)
        out = self.bn3(self.conv3(out))
        out = self.se(out)
        if self.downsample is not None: idt = self.downsample(x)
        return F.relu(out + idt, inplace=True)

class ResNet_DO(nn.Module):
    def __init__(self, layers, replace_stride_with_dilation):
        super().__init__()
        self.inplanes = 64; self.d = 1
        self.conv1 = nn.Conv2d(6, 64, 7, 2, 3, bias=False)
        self.bn1   = nn.BatchNorm2d(64)
        self.relu  = nn.ReLU(inplace=True)
        self.maxp  = nn.MaxPool2d(3, 2, 1)
        self.layer1 = self._make_layer(64,  layers[0])
        self.layer2 = self._make_layer(128, layers[1], stride=2,
                                       dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(256, layers[2], stride=2,
                                       dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(512, layers[3], stride=2,
                                       dilate=replace_stride_with_dilation[2])
    def _make_layer(self, planes, blocks, stride=1, dilate=False):
        down=None; prev_d=self.d
        if dilate: self.d *= stride; stride=1
        if stride!=1 or self.inplanes!=planes*Bottleneck.expansion:
            down = nn.Sequential(conv1(self.inplanes, planes*4, stride),
                                 nn.BatchNorm2d(planes*4))
        layers=[Bottleneck(self.inplanes, planes, stride, down, prev_d)]
        self.inplanes = planes*4
        for _ in range(1, blocks):
            layers.append(Bottleneck(self.inplanes, planes, dilation=self.d))
        return nn.Sequential(*layers)
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x))); x = self.maxp(x)
        low = self.layer1(x)
        x   = self.layer2(low); x = self.layer3(x); x = self.layer4(x)
        return {'low_level': low, 'out': x}

class ASPPConv(nn.Sequential):
    def __init__(self, i, o, r): super().__init__(
        nn.Conv2d(i, o, 3, padding=r, dilation=r, bias=False),
        nn.BatchNorm2d(o), nn.ReLU(inplace=True))
class ASPPPool(nn.Module):
    def __init__(self, i, o):
        super().__init__()
        self.avg = nn.AdaptiveAvgPool2d(1)
        self.conv= nn.Conv2d(i, o, 1, bias=False)
        self.gn  = nn.GroupNorm(1, o)
    def forward(self, x):
        y=self.avg(x); y=self.conv(y); y=self.gn(y); y=F.relu(y,inplace=True)
        return F.interpolate(y, x.shape[-2:], mode='bilinear', align_corners=False)
class ASPP(nn.Module):
    def __init__(self, in_c, rates=(12,24,36)):
        super().__init__()
        oc=256
        self.br = nn.ModuleList(
            [nn.Sequential(nn.Conv2d(in_c, oc, 1, bias=False),
                           nn.BatchNorm2d(oc), nn.ReLU(inplace=True))] +
            [ASPPConv(in_c, oc, r) for r in rates] +
            [ASPPPool(in_c, oc)])
        self.out = nn.Sequential(
            nn.Conv2d(len(self.br)*oc, oc, 1, bias=False),
            nn.BatchNorm2d(oc), nn.ReLU(inplace=True), nn.Dropout(0.1))
    def forward(self, x): return self.out(torch.cat([b(x) for b in self.br], 1))
class Head(nn.Module):
    def __init__(self, in_c, low_c, nc):
        super().__init__()
        self.low = nn.Sequential(conv1(low_c, 48), nn.BatchNorm2d(48), nn.ReLU(inplace=True))
        self.aspp= ASPP(in_c)
        self.cls = nn.Sequential(conv3(48+256, 256), nn.BatchNorm2d(256),
                                 nn.ReLU(inplace=True), conv1(256, nc))
    def forward(self, f):
        l = self.low(f['low_level'])
        h = self.aspp(f['out'])
        h = F.interpolate(h, l.shape[-2:], mode='bilinear', align_corners=False)
        return self.cls(torch.cat([l, h], 1))
class DeepLab(nn.Module):
    def __init__(self):
        super().__init__()
        self.back = ResNet_DO([3, 4, 23, 3], replace_stride_with_dilation=[False,True,True])
        self.head = Head(2048, 256, 3)
    def forward(self, x):
        sz = x.shape[-2:]; y = self.head(self.back(x))
        return F.interpolate(y, sz, mode='bilinear', align_corners=False)

model = DeepLab().to(device)

# -----------------------------------------------------------------------------
# 8) LOAD IMAGENET ResNet-101 WEIGHTS  (conv1 adapted 3→6)
# -----------------------------------------------------------------------------
from torchvision.models import resnet101, ResNet101_Weights
pre = resnet101(weights=ResNet101_Weights.IMAGENET1K_V2).state_dict()

# adapt conv1
w = pre['conv1.weight']                           # (64,3,7,7)
mean = w.mean(1, keepdim=True)                    # (64,1,7,7)
pre['conv1.weight'] = torch.cat([w, mean, mean, mean], 1)[:, :6]

# strip fc/avgpool to avoid size mismatch
pre = {k: v for k, v in pre.items() if not k.startswith('fc.')}

missing, unexpected = model.back.load_state_dict(pre, strict=False)
print(f"✔ loaded ImageNet weights -> missing={len(missing)}   SE params left random.")

# freeze backbone initially
for p in model.back.parameters(): p.requires_grad = False
for p in model.head.parameters(): p.requires_grad = True

# -----------------------------------------------------------------------------
# 9) DATALOADERS
# -----------------------------------------------------------------------------
BS, ACC, EPOCHS = 4, 2, 20
tr_ld = DataLoader(train_ds, BS, True,  num_workers=2, pin_memory=True)
va_ld = DataLoader(val_ds,   BS, False, num_workers=2, pin_memory=True)

# -----------------------------------------------------------------------------
# 10) OPTIMIZER  &  COSINE-WARM-RESTARTS (every 10 epochs)
# -----------------------------------------------------------------------------
opt = AdamW([
    {'params': model.head.parameters(),         'lr': 1e-3},
    {'params': model.back.layer4.parameters(),  'lr': 1e-5},
    {'params': model.back.layer3.parameters(),  'lr': 1e-6},
], weight_decay=1e-4)

steps_per_epoch = math.ceil(len(tr_ld) / ACC)
sched = CosineAnnealingWarmRestarts(
    opt, T_0=steps_per_epoch*10, T_mult=1, eta_min=1e-6)

scaler = GradScaler()
ema    = AveragedModel(model)        # exponential moving average

# -----------------------------------------------------------------------------
# 11) METRICS
# -----------------------------------------------------------------------------
mIoU = MulticlassJaccardIndex(3).to(device)
best = float('inf')

# -----------------------------------------------------------------------------
# 12) TRAIN + VALIDATE
# -----------------------------------------------------------------------------
for ep in range(1, EPOCHS+1):
    # stage-wise unfreeze
    if ep == 15:
        for p in model.back.layer4.parameters(): p.requires_grad = True
    if ep == 25:
        for p in model.back.layer3.parameters(): p.requires_grad = True

    model.train(); tot = 0.0; opt.zero_grad(set_to_none=True)
    for i, (x, y) in enumerate(tr_ld, 1):
        x, y = x.to(device), y.to(device)
        with autocast():
            out  = model(x); loss = loss_fn(out, y).mean() / ACC
        scaler.scale(loss).backward(); tot += loss.item() * ACC
        if i % ACC == 0 or i == len(tr_ld):
            scaler.step(opt); scaler.update(); opt.zero_grad(set_to_none=True)
        # cosine schedule per batch
        sched.step(ep - 1 + i / len(tr_ld))
        ema.update_parameters(model)
    tr_loss = tot / len(tr_ld)

    model.eval(); ema.module.eval()
    val_loss = 0.0; mIoU.reset()
    with torch.no_grad():
        for x, y in va_ld:
            x, y = x.to(device), y.to(device)
            p1 = ema.module(x)
            p2 = torch.flip(ema.module(torch.flip(x, [3])), [3])
            p3 = torch.flip(ema.module(torch.flip(x, [2])), [2])
            p  = (p1 + p2 + p3) / 3.0
            val_loss += loss_fn(p, y).mean().item()
            mIoU.update(p.argmax(1), y)
    val_loss /= len(va_ld); miou = mIoU.compute().mean()*100
    print(f"E{ep:02d}  Tr {tr_loss:.3f} | Va {val_loss:.3f} | mIoU {miou:.2f}%")

/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.6' (you have '2.0.4'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


▶ device: cuda
984 train   | 423 val
class-weights: [0.3799999952316284, 2.8429999351501465, 68.59100341796875]


/tmp/ipykernel_31/366659481.py:92: UserWarning: Argument(s) 'value, mask_value' are not valid for transform PadIfNeeded
  A.PadIfNeeded(SIZE[0], SIZE[1], border_mode=cv2.BORDER_CONSTANT,
/tmp/ipykernel_31/366659481.py:96: UserWarning: Argument(s) 'value, mask_value' are not valid for transform Rotate
  A.Rotate(30, border_mode=cv2.BORDER_CONSTANT, value=0, mask_value=0, p=.5),
Downloading: "https://download.pytorch.org/models/resnet101-cd907fc2.pth" to /root/.cache/torch/hub/checkpoints/resnet101-cd907fc2.pth
100%|██████████| 171M/171M [00:00<00:00, 222MB/s] 


✔ loaded ImageNet weights -> missing=132   SE params left random.


/tmp/ipykernel_31/366659481.py:326: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
/tmp/ipykernel_31/366659481.py:348: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


E01  Tr 0.555 | Va 0.393 | mIoU 65.41%
E02  Tr 0.396 | Va 0.362 | mIoU 68.85%
E03  Tr 0.352 | Va 0.336 | mIoU 69.58%
E04  Tr 0.341 | Va 0.334 | mIoU 72.63%
E05  Tr 0.324 | Va 0.337 | mIoU 67.48%
E06  Tr 0.310 | Va 0.300 | mIoU 69.73%
E07  Tr 0.309 | Va 0.345 | mIoU 70.55%
E08  Tr 0.297 | Va 0.314 | mIoU 72.45%
E09  Tr 0.279 | Va 0.300 | mIoU 70.41%
E10  Tr 0.284 | Va 0.280 | mIoU 72.87%
E11  Tr 0.288 | Va 0.301 | mIoU 75.05%
E12  Tr 0.281 | Va 0.287 | mIoU 73.70%
E13  Tr 0.265 | Va 0.277 | mIoU 75.70%
E14  Tr 0.267 | Va 0.292 | mIoU 73.31%
E15  Tr 0.250 | Va 0.257 | mIoU 73.31%
E16  Tr 0.247 | Va 0.271 | mIoU 77.01%
E17  Tr 0.240 | Va 0.261 | mIoU 73.34%
E18  Tr 0.238 | Va 0.254 | mIoU 74.47%
E19  Tr 0.242 | Va 0.255 | mIoU 71.73%
E20  Tr 0.230 | Va 0.255 | mIoU 73.47%


Your latest edge-aware mirror pipeline shows a clear, measurable step forward.
Starting from an mIoU of roughly 65 % on the first epoch (after the ImageNet-initialised ResNet-101 was adapted to six channels) the network climbs quickly: within four epochs the combination of deeper backbone, newly added SE blocks and the Sobel edge channel pushes validation mIoU to the low-70s. The first cosine warm-restart at epoch 10 nudges the optimiser out of its initial basin and you subsequently crest 77 % at epoch 16 – a net gain of almost 12 percentage points over the opening score and about 9 pp better than the earlier 5-band ResNet-50 model you were using.

Why did those changes help? The 101-layer backbone extracts stronger high-level semantics than the 50-layer version, while the SE squeeze-and-excite gates let the model emphasise vegetation features and down-weight soil or illumination artefacts. Feeding an explicit Sobel magnitude map adds a crisp boundary cue that benefits the small, filament-like weed structures this dataset contains. Mirror test-time augmentation (horizontal and vertical flips) smooths prediction noise by majority-voting three independent views. Finally, cosine warm-restarts every ten epochs allow the head (and, once unfrozen, the upper residual stages) to re-explore learning-rate space instead of flattening out prematurely, which explains the renewed drop in loss and lift in accuracy around epoch 11–16.

The plateau that appears after epoch 16 is typical: once the restart cycle finishes the optimiser again eases toward a flat minimum and improvements taper. Your validation loss and the gap between training and validation losses remain small, so heavy over-fit is not yet an issue, but the mild oscillation in mIoU suggests the learning rate could decay more gently; adopting a schedule with progressively longer restart periods or a cosine anneal without restarts for the backbone while keeping restarts only for the head might squeeze out a little extra performance.

A few opportunities remain. Random-box CutMix with a higher probability or additional colour-jitter / geometric warps would help inject still more variety and curb the slight over-fitting trend that starts around epoch 14. Replacing the single Sobel magnitude with two oriented Sobel channels, or with a learned HED or Canny edge map, often gives another one or two points on small weed datasets. On the loss side, a boundary-aware term such as Lovász-Softmax or BoundaryLoss layered on top of your CE + Dice + Focal cocktail can directly reward crisp outlines and typically produces a further one-point mIoU bump. Finally, introducing modest multi-scale inference (for example 0.75× and 1.25× scales in addition to the current flips) is cheap at batch-size 1 and consistently adds a percent or two.

Overall, I would rate the current incarnation at about 8 / 10 for segmentation craftsmanship on this dataset. The data pipeline is strong, the architecture choice is solid and modern tricks like EMA and cosine restarts are in place. The remaining gap to the very top performers (hovering around 80 % on PhenoBench) is now mainly about refining boundary handling and exploiting more scale/edge diversity rather than wholesale architectural change, though transformer-hybrid heads (e.g. SegNeXt, HRFormer) could eventually offer an extra push once you have exhausted CNN-based gains.